In [1]:
%%javascript
$('#appmode-leave').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Determination of the Rate Law \#1
The rates of thermolysis of a variety of $cis-$ and $trans-$azoalkanes have been measured over a range of temperatures to settle a controversy concerning the mechanism of the reaction. 
In ethanol, an unstable $cis-$azoalkane decomposed at a rate that can be followed by observing the $N_2$ evolution, and this can be used to determine the rate constant of the reaction. 
Perform a virtual series of such experiments at different temperatures and calculate the enthalpy, entropy and the Gibbs free energy of activation at -20$^\circ$C.


### Instructions:

- Use the slide bar below to select temperature at which you perform the virtual experiment, 
- Click `Perform measurement` to run the virtual experiment and obtain the result of the experiment,
- Click `Download CSV` to export the complete data set for all the experiments as a CSV file.

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

In [3]:
# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin6"]


In [4]:
# delete existing result file and setup rng
if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
    os.remove(os.path.join(os.getcwd(), "results.csv"))

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )


In [5]:
out_R = ipw.Label(value="Click Calculate.")
out_P = ipw.Output()
out_L = ipw.Output()

result = "Rate constant (s$^{-1}$)"

with out_L:
    display(Markdown("[Download CSV](./results.csv)"))
    
def calc(btn):
    lnK = -params["dH"] / 8.3145 / temp.value + params["dS"] / 8.3145
    R = math.exp(lnK)
    
    # Random error
    dR = R * params["error"].get("random", 0) * (0.5 - random.random()) * 2 
    # Systematic error
    dR = dR + R * params["error"].get("systematic", 0)
    out_R.value = f"{R + dR:.6e}"
    out_P.clear_output()
    try:
        res = pd.read_csv("results.csv") 
    except FileNotFoundError:
        res = pd.DataFrame(columns=["Temperature (K)",
                                    result])
    res = res.append({"Temperature (K)": temp.value, 
                      result: R + dR}, 
                     ignore_index=True)
    res.to_csv("results.csv", index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
        os.remove(os.path.join(os.getcwd(), "results.csv"))
    res = pd.DataFrame(columns=["Temperature (K)",
                                result])

    res.to_csv("results.csv", index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(50))

temp = ipw.FloatSlider(value=params["temp"], min=100, max=300)

btn_reset = ipw.Button(description="Restart Laboratory", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)


btn_calc = ipw.Button(description="Perform measurement", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

rows = []
rows.append(ipw.HBox([ipw.Label(value=f"Temperature:", 
                                layout=ipw.Layout(width="250px")),
                      temp,
                      ipw.Label(value="(K)")]))

rows.append(ipw.HBox([ipw.Label(value="Rate constant:", 
                                layout=ipw.Layout(width="250px")), 
                      out_R,
                      ipw.Label(value="$(s^{-1})$")]))

rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))
rows.append(ipw.HBox([out_P]))
reset(btn_reset)
ipw.VBox(rows)

-7.142295433834313
10644.055565578206 260.0
33.79637981838956
